# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved.
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server.

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode


from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
import numpy as np
from keras.layers import Conv1D, LSTM, Dense, TimeDistributed, Embedding, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Bidirectional
import random

def load_data(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        words = file.read().splitlines()
    return words

def simulate_missing_letters(word, p_missing=0.45):
    """
    Randomly replace letters with '0' to simulate missing letters,
    with p_missing chance for each letter.
    """
    simulated_word = ''
    for char in word:
        simulated_word += '0' if random.random() < p_missing else char
    return simulated_word

def preprocess_data(words, char_to_int, max_word_length, p_missing=0.45):
    sequences = []
    targets = []
    for word in words:
        word_with_missing = simulate_missing_letters(word, p_missing)
        sequences.append([char_to_int[char] if char in char_to_int else char_to_int['0'] for char in word_with_missing])
        targets.append([char_to_int[char] for char in word])
    sequences = pad_sequences(sequences, maxlen=max_word_length, padding='post')
    targets = pad_sequences(targets, maxlen=max_word_length, padding='post')
    return sequences, targets

file_path = '/content/words_250000_train.txt'
words = load_data(file_path)
chars = sorted(list(set(''.join(words) + '0')))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
max_word_length = 29

X, y = preprocess_data(words, char_to_int, max_word_length)
y = to_categorical(y, num_classes=len(chars))

model = Sequential()
model.add(Embedding(input_dim=len(chars), output_dim=128, trainable=True))
model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(Bidirectional(LSTM(256, return_sequences=True, kernel_regularizer=l2(0.001))))
model.add(Dropout(0.4))
model.add(TimeDistributed(Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min', restore_best_weights=True)

model.fit(X, np.array(y), validation_split=0.2, epochs=20, batch_size=128, callbacks=[early_stopping])

Epoch 1/20
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 43s 22ms/step - accuracy: 0.8224 - loss: 0.7336 - val_accuracy: 0.8786 - val_loss: 0.4425
Epoch 2/20
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 30s 21ms/step - accuracy: 0.8869 - loss: 0.3980 - val_accuracy: 0.8811 - val_loss: 0.4373
Epoch 3/20
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 30s 21ms/step - accuracy: 0.8903 - loss: 0.3838 - val_accuracy: 0.8839 - val_loss: 0.4318
Epoch 4/20
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 41s 21ms/step - accuracy: 0.8918 - loss: 0.3767 - val_accuracy: 0.8844 - val_loss: 0.4365
Epoch 5/20
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 41s 21ms/step - accuracy: 0.8931 - loss: 0.3717 - val_accuracy: 0.8852 - val_loss: 0.4357
Epoch 6/20
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 41s 21ms/step - accuracy: 0.8943 - loss: 0.3669 - val_accuracy: 0.8855 - val_loss: 0.4330
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 3.


In [28]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from collections import defaultdict, Counter

def predict_word(model, word_with_missing, char_to_int, int_to_char, max_word_length, guessed_letters):
    print("LSTM")
    word_encoded = [char_to_int[char] if char in char_to_int else char_to_int['0'] for char in word_with_missing]
    word_padded = pad_sequences([word_encoded], maxlen=max_word_length, padding='post')

    prediction = model.predict(word_padded)
    predict_vector = prediction[0]

    for i, char in enumerate(word_with_missing):
        if char == '0':
            probabilities = predict_vector[i]
            sorted_indices = np.argsort(-probabilities)
            for idx in sorted_indices:
                predicted_char = int_to_char[idx]
                if predicted_char != '0' and predicted_char not in guessed_letters:
                    return predicted_char
    return None  # Return None if all characters are guessed or no suitable character is found

In [29]:
class Node:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class DAWG:
    def __init__(self):
        self.root = Node()

    def add_word(self, word):
        current = self.root
        for letter in word:
            if letter not in current.children:
                current.children[letter] = Node()
            current = current.children[letter]
        current.is_end_of_word = True

    # Search for words with similar patterns using DFS
    def generate_possible_words(self, node, partial_word):
        possible_words = []

        def dfs(current_node, current_prefix):
            if current_node.is_end_of_word:
                possible_words.append(current_prefix)
            if len(current_prefix) < len(partial_word):
                next_letter_index = len(current_prefix)
                next_letter = partial_word[next_letter_index]
                for letter, child_node in current_node.children.items():
                    if next_letter == '_' or next_letter == letter:
                        dfs(child_node, current_prefix + letter)

        dfs(node, '')
        return possible_words

In [30]:
class NGramModel:
    def __init__(self, n):
        self.n = n
        self.ngrams = defaultdict(lambda: defaultdict(int))
        self.context_counts = defaultdict(int)
    def train(self, words):
        """
        Train the N-Gram model using a list of words.
        """
        for word in words:
            padded_word = ['<s>'] * (self.n - 1) + list(word) + ['</s>']

            # Extract n-grams and their contexts
            for i in range(len(padded_word) - self.n + 1):
                context = tuple(padded_word[i:i + self.n - 1])
                next_char = padded_word[i + self.n - 1]
                self.ngrams[context][next_char] += 1
                self.context_counts[context] += 1
    def predict(self, context):
        """
        Predict the next character based on the given context.
        Returns a dictionary of characters with their probabilities.
        """
        context = tuple(context[-(self.n - 1):])  # Consider only the last n-1 characters as context
        total_count = self.context_counts[context]
        if total_count == 0:
            return {}

        # Calculate probabilities for the next character
        probabilities = {char: count / total_count for char, count in self.ngrams[context].items()}
        return probabilities

In [31]:
def predict_next_letter_dawg_ngram(partial_word, guessed_letters, dawg, ngram_model):
    """
    Predict the next letter by combining DAWG and N-Gram models.
    """
    word = partial_word.replace(" ", "")
    word = ''.join([letter if letter in guessed_letters else '_' for letter in word])
    if word.count('_') <= 0.4*(len(word)):
        word_with_missing = word.replace('_', '0')
        return predict_word(model, word_with_missing, char_to_int, int_to_char, max_word_length, guessed_letters)
    # Use DAWG to generate possible words matching the pattern
    possible_words = dawg.generate_possible_words(dawg.root, word)

    # Calculate letter frequencies from possible words using DAWG
    letter_frequency = defaultdict(int)
    for word in possible_words:
        for letter in word:
            if letter not in guessed_letters:
                letter_frequency[letter] += 1

    # Use N-Gram model to predict the next character based on the context
    ngram_context = [char for char in partial_word if char != ' ' and char in guessed_letters][-ngram_model.n+1:]
    ngram_probabilities = ngram_model.predict(ngram_context)

    # Combine DAWG frequencies with N-Gram probabilities
    combined_scores = defaultdict(float)
    for letter, freq in letter_frequency.items():
        ngram_prob = ngram_probabilities.get(letter, 0)
        combined_scores[letter] = freq * (1 + ngram_prob)  # Combining frequency and probability

    # Sort letters by combined score in descending order
    sorted_letters = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)

    for letter, _ in sorted_letters:
        if letter not in partial_word:
            return letter

    word = partial_word.replace(" ", "")
    word = ''.join([letter if letter in guessed_letters else '_' for letter in word])
    word_with_missing = word.replace('_', '0')
    return predict_word(model, word_with_missing, char_to_int, int_to_char, max_word_length, guessed_letters)

In [32]:
ngram_model = NGramModel(n=5)
ngram_model.train(words)

hangman_dawg = DAWG()
for word in words:
    hangman_dawg.add_word(word)

In [3]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = '/content/words_250000_train.txt'
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

        self.current_dictionary = []

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word):

        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        clean_word = word[::2]

        # find length of passed word
        len_word = len(clean_word)

        partial_word = clean_word
        next_letter = next_letter = predict_next_letter_dawg_ngram(partial_word, self.guessed_letters, hangman_dawg, ngram_model)

        guess_letter = next_letter

        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object.
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [4]:
api = HangmanAPI(access_token="004e2ec4113cdd5c9e2d739cd8e4a7",timeout=2000)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [35]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

Successfully start a new game! Game ID: 1d1ef862e16c. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '1d1ef862e16c', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ _ _ _ _ _ '}
Guessing letter: i
Sever response: {'game_id': '1d1ef862e16c', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ i _ _ _ _ _ _ _ _ _ _ _ _ '}
Guessing letter: s
Sever response: {'game_id': '1d1ef862e16c', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ i _ _ _ _ _ _ _ _ _ _ _ _ '}
Guessing letter: n
Sever response: {'game_id': '1d1ef862e16c', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ i _ _ _ _ _ _ n _ _ _ _ _ '}
Guessing letter: l
Sever response: {'game_id': '1d1ef862e16c', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ i _ _ _ _ _ _ n _ _ _ _ _ '}
Guessing letter: t
Sever response: {'game_id': '1d1ef862e16c', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ i _ _ _ _ _ _ n _ _ _ _ _ '}
Guessing letter: a
Se

In [8]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status()
total_practice_successes

523

In [9]:
total_practice_runs

1439

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)

    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Streaming output truncated to the last 5000 lines.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
LSTM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
LSTM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Playing  296  th game
LSTM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
LSTM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
LSTM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
LSTM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
LSTM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
LSTM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
LSTM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Playing  297  th game
Playing  298  th game
Playing  299  th game
Playing  300  th game
LSTM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
LSTM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
LSTM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
LSTM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
LSTM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Playing  301  th game
LSTM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
LSTM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
LSTM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Playing  302  th game
LSTM
1/1 ━━━━━━━━━━━━━━━━━

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [7]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)
print('total recorded runs', total_recorded_runs)
print('total recorded successes', total_recorded_successes)

overall success rate = 0.409
total recorded runs 900
total recorded successes 368
